# Data-Merge
* 전처리한 각 파일을 필요에 따라 merge

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 
%config InlineBackend.figure_format = 'retina' 

In [2]:
# 전처리 된 모든 데이터 불러오기

customer = pd.read_csv('customer.csv')
shop_product = pd.read_csv('shop_df.csv')
out_product = pd.read_csv('shopno_df.csv')
classifi_product = pd.read_csv('productvar.csv')

# 쇼핑업종 관련 데이터 생성
**고객Demo × 쇼핑업종 데이터 × 상품분류 데이터**
* 고객데이터와 쇼핑업종 데이터 merge
* 구매금액과 구매수량을 곱하여 '총쇼핑소비금액'이라는 새로운 컬럼 생성
* 상품분류 데이터 merge 시킨 후 필요없는 컬럼 삭제

In [3]:
# merge 함수 생성

def cus_shop_pro(cus_df, shop_df, prod_df):
    
    customer_shop = pd.merge(cus_df, shop_df, on='ID', how='inner')
    
    # copy 후 새로운 컬럼 생성
    customer_shop1 = customer_shop.copy()
    customer_shop1['총_쇼핑소비금액'] = customer_shop1['구매금액']*customer_shop1['구매수량']
    
    # 각 고객별 총쇼핑소비금액 df 생성
    customer_sum_shop = pd.DataFrame(customer_shop1.groupby(['ID', '성별', '연령대', '시도', '시군구'])['총_쇼핑소비금액'].sum().reset_index())
    
    # 위의 df와 쇼핑업종df 다시 merge (고객 쇼핑업종 총금액 df)
    df1 = pd.merge(customer_sum_shop, shop_df, on='ID', how='inner')
    
    df2 = prod_df
    
    
    # df1과 df2 merge를 위해 key값 생성
    
    df1['업종'] = df1['업종'].astype('string')
    df1['상품소분류코드'] = df1['상품소분류코드'].astype('string')
    df1['key'] = df1['업종']+df1['상품소분류코드']
    
    df2['업종'] = df2['업종'].astype('string')
    df2['상품소분류코드'] = df2['상품소분류코드'].astype('string')
    df2['key'] = df2['업종']+df2['상품소분류코드']
    
    # merge 후 필요한 컬럼만 골라내기
    df12 = pd.merge(df1, df2, on='key', how='left')
    real_df = df12[['ID', '성별', '연령대', '시도', '시군구', '총_쇼핑소비금액', '구매일자', '구매시간','구매금액', '구매수량', '업종_x', '상품소분류코드_x', 'key', '소분류명','중분류명', '대분류명']]
    real_df = real_df.rename(columns={'업종_x' : '업종', '상품소분류코드_x': '상품소분류코드'})
    real_df = real_df.drop(columns=['key', '상품소분류코드'], axis=1)
    
    return real_df

In [4]:
df_shop = cus_shop_pro(customer, shop_product, classifi_product)
df_shop

,ID,성별,연령대,시도,시군구,총_쇼핑소비금액,구매일자,구매시간,구매금액,구매수량,업종,소분류명,중분류명,대분류명
0,1,남성,60대,서울특별시,강동구,3899130,2015-12-16,17,28000,1,백화점,수입주방,주방용품,가정
1,1,남성,60대,서울특별시,강동구,3899130,2015-05-13,14,15000,1,백화점,수입주방,주방용품,가정
2,1,남성,60대,서울특별시,강동구,3899130,2015-08-01,19,166400,1,백화점,N/B골프의류,골프용품,스포츠
3,1,남성,60대,서울특별시,강동구,3899130,2015-12-28,12,166000,1,백화점,란제리,란제리/내의,여성의류
4,1,남성,60대,서울특별시,강동구,3899130,2015-05-13,12,1000,2,백화점,어묵,젓갈/반찬,식품
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447065,19998,남성,40대,경상남도,김해시,321700,2015-12-26,17,15000,1,대형마트,남성병행수입,병행수입,남성의류편집
3447066,19998,남성,40대,경상남도,김해시,321700,2015-12-26,14,40000,4,대형마트,일반화장품,기초화장품,브랜드화장품
3447067,19998,남성,40대,경상남도,김해시,321700,2015-12-26,14,8000,8,대형마트,일반화장품,기초화장품,브랜드화장품
3447068,19998,남성,40대,경상남도,김해시,321700,2015-12-26,14,500,5,대형마트,일반화장품,기초화장품,브랜드화장품


# 쇼핑외업종 관련 데이터 생성
**고객 Demo 데이터 × 쇼핑외업종 데이터**
* 고객데이터와 쇼핑외업종 데이터 merge
* 쇼핑외업종 데이터의 이용금액과 이용건수를 곱하여 총쇼핑외금액 컬럼 생성

In [5]:
# merge 함수 생성

def cus_shopno_mer(cus_df, shopno_df):
    
    # 고객데이터와 쇼핑외업종 데이터 merge
    customer_out = pd.merge(cus_df, shopno_df, on='ID', how='inner')
    
    # 데이터 copy하여 새 컬럼 생성
    customer_out1 = customer_out.copy()
    customer_out1['총_쇼핑외금액'] = customer_out1['이용금액'] * customer_out1['이용건수']
    
    # 각 고객별 총쇼핑외금액 df 생성
    customer_sum_out = pd.DataFrame(customer_out1.groupby(['ID', '성별', '연령대', '시도', '시군구'])['총_쇼핑외금액'].sum().reset_index())
    
    customer_out = pd.merge(customer_sum_out, out_product, on='ID', how='inner')
    
    return customer_out

In [6]:
df_shopno = cus_shopno_mer(customer, out_product)
df_shopno

,ID,성별,연령대,시도,시군구,총_쇼핑외금액,쇼핑외업종,이용월,이용금액,이용건수
0,1,남성,60대,서울특별시,강동구,2407864,면세점,2015-02-01,270518,3
1,1,남성,60대,서울특별시,강동구,2407864,카페,2015-12-01,9200,1
2,1,남성,60대,서울특별시,강동구,2407864,영화관,2015-12-01,7500,1
3,1,남성,60대,서울특별시,강동구,2407864,면세점,2015-12-01,69887,1
4,1,남성,60대,서울특별시,강동구,2407864,영화관,2015-12-01,14000,1
...,...,...,...,...,...,...,...,...,...,...
166972,19987,여성,20대,서울특별시,송파구,59000,영화관,2015-11-01,9000,1
166973,19992,남성,40대,서울특별시,강북구,1500,패스트푸드,2015-12-01,1500,1
166974,19993,남성,60대,부산광역시,사상구,50956,면세점,2015-12-01,50956,1
166975,19994,남성,60대,경기도,평택시,646372,면세점,2015-12-01,161593,4


# 최종 데이터 저장
* 분석시 사용할 데이터인 df_shop(쇼핑업종데이터)와 df_shopno(쇼핑외업종데이터) 저장

In [7]:
df_shop.to_csv('df_shop.csv', index=False)
df_shopno.to_csv('df_shopno.csv', index=False)